# Tutorial : introduction to MLFlow

This first application introduces the basic concepts of `MLFlow`. The goal is to predict the income class of individuals using a sample data from the US Census and a Random Forest classifier using the popular [scikit-learn](https://scikit-learn.org/stable/) machine learning `Python` library. 

We first illustrate how we would perform the training and fine-tuning of the model in a traditional way. Then, we show how we can integrate it as an **MLflow experiment**, so as to **log** relevant parameters and metrics in `MLflow`'s **tracking server** and visualize them in the UI. Finally, we illustrate how selected models can transition from the tracking server to the **model registry**, and how they can then be used from there to perform inference on new data points.

In [20]:
import os
from pprint import pprint
import json

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
import mlflow
import mlflow.sklearn
import mlflow.pyfunc

In [21]:
SEED = 0

## Data preprocessing

For this application, we'll use a classical dataset extracted from the 1994 US Census bureau data. The goal is to determine whether a person makes over $50K a year ('>50K') or less ('<=50K') using sociodemographic characteristics on the selected individuals. As the available variables are generally self-explanatory, we won't describe the data much, but more information on them can be found in the original [Kaggle challenge](https://www.kaggle.com/datasets/uciml/adult-census-income).

In [22]:
url_data = "https://minio.lab.sspcloud.fr/projet-formation/diffusion/mlops/data/adult-census-us.csv"
df_census = pd.read_csv(url_data)

A common practice in machine learning projects is to start by setting a fraction of the data aside as a **test dataset**. This data will be used at the very end of the project in order to properly evaluate the generalization performance of our selected algorithm, i.e. how it would perform on new unseen data. The rest of the data (**training dataset**) will be used to train the algorithms and compare their performance. Without this step, we are at risk of overfitting our models on the available data so that our evaluation metrics would no longer properly estimate the generalization error.

In [23]:
df_train, df_test = train_test_split(df_census, test_size=0.2, random_state=SEED)

In [24]:
df_train.sample(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country,class
19570,2,Private,188503,Some-college,10,Never-married,Adm-clerical,Unmarried,White,Female,2,0,1,United-States,<=50K
12908,4,NaN,299494,11th,7,Married-civ-spouse,NaN,Husband,White,Male,1,0,2,United-States,<=50K
26426,2,Private,105021,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,3,United-States,>50K
10195,0,Private,122048,HS-grad,9,Never-married,Machine-op-inspct,Not-in-family,White,Female,0,0,2,United-States,<=50K
30472,2,Private,176286,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,2,United-States,<=50K


The goal is to predict the income class, so we must first exclude it from the training data. As this variable consists in string-encoded categories, we must encode it in a numerical format to be able to feed it to a machine learning model. A common practice for ordinal data (data for which an order exists, such as income class) is to encode labels as subsequent integers starting at 0, a technique known as **label encoding**.

In [25]:
le = LabelEncoder()

X_train = df_train.drop(columns="class")
y_train = le.fit_transform(df_train["class"].values)

In [26]:
y_train

array([0, 0, 0, ..., 1, 1, 0])

These new integer-encoded categories can naturally be mapped to the original values of the variable.

In [27]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [28]:
np.array([le.classes_[i] for i in y_train])

array(['<=50K', '<=50K', '<=50K', ..., '>50K', '>50K', '<=50K'],
      dtype='<U5')

In [29]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39073 entries, 22729 to 2732
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             39073 non-null  int64 
 1   workclass       36830 non-null  object
 2   fnlwgt          39073 non-null  int64 
 3   education       39073 non-null  object
 4   education-num   39073 non-null  int64 
 5   marital-status  39073 non-null  object
 6   occupation      36821 non-null  object
 7   relationship    39073 non-null  object
 8   race            39073 non-null  object
 9   sex             39073 non-null  object
 10  capitalgain     39073 non-null  int64 
 11  capitalloss     39073 non-null  int64 
 12  hoursperweek    39073 non-null  int64 
 13  native-country  38404 non-null  object
dtypes: int64(6), object(8)
memory usage: 4.5+ MB


These general information show that thousands of observations are missing for some variables. To avoid wasting data and since these might not be missing-at-random, we'll impute values for the missing ones :
- for numerical variables, we'll impute the median of the variable
- for categorical variables, we'll impute the mode, i.e. the most frequent category in the data

As previously, string-encoded categorical variables must also be converted to some form of numerical data. We'll use the same encoding strategy as the one used to encode the target variable.

So as to make all these steps as reproducible as possible, we formalize them as a `scikit-learn` `Pipeline` object. More information on their justification and the way there are used can be found in the [documentation](https://scikit-learn.org/stable/modules/compose.html).

In [30]:
median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mode_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

categorical_transformer = make_pipeline(mode_imputer, ordinal_encoder)

preprocessor = ColumnTransformer(
    transformers=[
        ("numerical", median_imputer, make_column_selector(dtype_include=np.int64)),
        ("categorical", categorical_transformer, make_column_selector(dtype_include=object))
    ], remainder="passthrough"
)

As most `scikit-learn` objects, the pipeline must first `fit` the data (e.g. compute the most frequent value or median). Then, we can use it to `transform` the data. The resulting object is a `NumPy` array with the same structure as the original data. It is not very useful per se, but it shows us that the categorical variables are indeed transformed into numerical values. This numerical array can then be fed to a machine learning model to train it.

In [13]:
preprocessor.fit_transform(X_train)

array([[0.00000e+00, 1.17372e+05, 7.00000e+00, ..., 4.00000e+00,
        1.00000e+00, 3.80000e+01],
       [2.00000e+00, 3.57720e+05, 1.10000e+01, ..., 4.00000e+00,
        0.00000e+00, 3.80000e+01],
       [4.00000e+00, 2.02242e+05, 9.00000e+00, ..., 4.00000e+00,
        1.00000e+00, 3.80000e+01],
       ...,
       [2.00000e+00, 3.44624e+05, 1.00000e+01, ..., 4.00000e+00,
        1.00000e+00, 3.80000e+01],
       [3.00000e+00, 1.04489e+05, 1.30000e+01, ..., 4.00000e+00,
        1.00000e+00, 3.80000e+01],
       [0.00000e+00, 1.86925e+05, 1.00000e+01, ..., 4.00000e+00,
        1.00000e+00, 3.80000e+01]])

## Training models : the classical way

In order to really understand why the MLOps approach is desirable, we must first get an idea of how we would train our model without it, i.e. the "classical" way. The workflow we are trying to achieve is best described by the following figure from the [scikit-learn documentation](https://scikit-learn.org/stable/).

![](img/grid_search_workflow.png){fig-align="center" height=300}

Using the training data, we want to train a model so as to get the best generalization performance, i.e. minimize the prediction error on unseen data. To do so, we have to **fine-tune** the **hyperparameters** of our model, i.e. find the combination of **hyperparameters** that provide the best performance. In order to avoid **overfitting** when doing so, we use a procedure called **cross-validation** (described in details [here](https://scikit-learn.org/stable/modules/cross_validation.html)). When we have found the optimal set of hyperparameters, we use the model train with those for a final evaluation on the test set.

However, we'll see that what's missing in this workflow is a proper way to **track experiments** (data used for training, environment, metrics...), which is absolutely essential in machine learning projects, where we generally want to compare many configurations before selecting a final model.

In [14]:
rf_clf = RandomForestClassifier(random_state=SEED)

pipe_rf = Pipeline([
    ('preprocessor', preprocessor), 
    ('classifier', rf_clf)
])

Although the hyperparameters provided natively by `scikit-learn` are usually good defaults, we will of course want to check whether we can improve the performance further by **fine-tuning** the relevant hyperparameters. 

In [15]:
param_grid = {
    "classifier__n_estimators": [50, 100, 200],
    "classifier__max_leaf_nodes": [5, 10, 50]
}


pipe_gscv = GridSearchCV(pipe_rf, param_grid=param_grid, 
                         scoring=["accuracy", "precision", "recall", "f1"],
                         refit="f1",
                         cv=5, n_jobs=5, verbose=1)

In [16]:
pipe_gscv.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('numerical',
                                                                         SimpleImputer(strategy='median'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7efc139c4df0>),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ordinalencoder',
                                                                                          OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                         unknown_value=-1))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7efc139c4c70>)])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=0))]),
             n_jobs=5,
             param_grid={'classifier__max_leaf_nodes': [5, 10, 50],
                         'classifier__n_estimators': [50, 100, 200]},
             refit='f1', scoring=['accuracy', 'precision', 'recall', 'f1'],
             verbose=1)

In [17]:
gscv_results = pd.DataFrame(pipe_gscv.cv_results_)
gscv_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__max_leaf_nodes,param_classifier__n_estimators,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,...,std_test_recall,rank_test_recall,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0.618934,0.010013,0.058359,0.002880,5,50,"{'classifier__max_leaf_nodes': 5, 'classifier_...",0.827639,0.833781,0.832118,...,0.013073,7,0.541993,0.545963,0.543811,0.557421,0.528157,0.543469,0.009356,7
1,1.098869,0.017184,0.093265,0.004500,5,100,"{'classifier__max_leaf_nodes': 5, 'classifier_...",0.825080,0.831734,0.827895,...,0.007155,8,0.514732,0.536809,0.523557,0.530239,0.517448,0.524557,0.008130,8
2,2.127793,0.079194,0.151213,0.004600,5,200,"{'classifier__max_leaf_nodes': 5, 'classifier_...",0.823800,0.828279,0.825848,...,0.005810,9,0.503067,0.520714,0.514449,0.513224,0.512221,0.512735,0.005667,9
3,0.701116,0.012558,0.062202,0.003840,10,50,"{'classifier__max_leaf_nodes': 10, 'classifier...",0.831862,0.841331,0.837876,...,0.010626,4,0.567763,0.595828,0.590894,0.574122,0.583145,0.582351,0.010351,4
4,1.228420,0.007473,0.099676,0.003677,10,100,"{'classifier__max_leaf_nodes': 10, 'classifier...",0.831094,0.840691,0.837236,...,0.009021,5,0.564931,0.591401,0.587281,0.575057,0.577098,0.579154,0.009374,5


In [18]:
pipe_gscv.best_params_

{'classifier__max_leaf_nodes': 50, 'classifier__n_estimators': 50}

In [22]:
if not os.path.exists("models/"):
    os.makedirs("models/")
joblib.dump(pipe_gscv, 'models/pipeline_train_model_20230118.joblib')

['models/pipeline_train_model_20230118.joblib']

## Training models : the MLFlow way

### Configuration

In [30]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://minio.lab.sspcloud.fr"

In [39]:
# Automatic discovery : if MLFlow has been launched before Jupyter/VSCode
print(os.environ["MLFLOW_TRACKING_URI"])

https://user-avouacr-620621.user.lab.sspcloud.fr


In [12]:
# Manual configuration : if MLFlow has been launched after Jupyter/VSCode
# os.environ["MLFLOW_TRACKING_URI"] = "copy_uri_from_mlflow_service_README_here"

### Training and tracking

In [40]:
def train_model_with_mlflow_tracking(mlflow_experiment_name,
                                     n_estimators,
                                     max_leaf_nodes):
    # Set up MLFlow context
    mlflow.set_experiment(experiment_name=mlflow_experiment_name)
    
    with mlflow.start_run():
        
        # Training step
        rf_clf = RandomForestClassifier(n_estimators=n_estimators,
                                        max_leaf_nodes=max_leaf_nodes,
                                        random_state=SEED)
        pipe_rf = Pipeline([
            ('preprocessor', preprocessor), 
            ('classifier', rf_clf)
        ])
        pipe_rf.fit(X_train, y_train)
        y_train_pred = pipe_rf.predict(X_train)
        
        # Compute fit metrics
        accuracy = accuracy_score(y_train_pred, y_train)
        precision = precision_score(y_train_pred, y_train)
        recall = recall_score(y_train_pred, y_train)
        f1 = f1_score(y_train_pred, y_train)
        
        # Track training data
        mlflow.log_param("data_url", url_data)
        
        # Track hyperparameters
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_leaf_nodes", max_leaf_nodes)
        
        # Track fit metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1", f1)

        # Track model binary
        mlflow.sklearn.log_model(pipe_rf, "model")

In [41]:
# Grid search
n_estimators_range = [50, 100, 200]
max_leaf_nodes_range = [5, 10, 50]

for n_estimator in n_estimators_range:
    for max_leaf_nodes in max_leaf_nodes_range:
        train_model_with_mlflow_tracking(
            mlflow_experiment_name="tutorial-mlflow",
            n_estimators=n_estimator,
            max_leaf_nodes=max_leaf_nodes
        )

### Using the model registry

Ajouter un premier modèle

In [34]:
def print_scores(y_pred, y):
    accuracy = accuracy_score(y_train_pred, y_train)
    f1 = f1_score(y_train_pred, y_train)

    print(f"""
    accuracy : {accuracy}
    f1_score : {f1}
    """)

In [42]:
model_name = "20230118_rf_census"
version = 1

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{version}"
)

y_train_pred = model.predict(X_train)
print_scores(y_train_pred, y_train)

2023/01/25 10:28:30 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 2.2.1, required: cloudpickle==2.2.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.



    accuracy : 0.8513551557341387
    f1_score : 0.6448575272104684
    


Tagguer le modèle en prod

In [36]:
model_name = "20230118_rf_census"
stage = 'Production'

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{stage}"
)

y_train_pred = model.predict(X_train)
print_scores(y_train_pred, y_train)

2023/01/25 10:20:20 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 2.2.1, required: cloudpickle==2.2.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.



    accuracy : 0.8513551557341387
    f1_score : 0.6448575272104684
    


Ajouter un second modèle

In [37]:
model_name = "20230118_rf_census"
version = 2

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{version}"
)

y_train_pred = model.predict(X_train)
print_scores(y_train_pred, y_train)

2023/01/25 10:20:23 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 2.2.1, required: cloudpickle==2.2.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.



    accuracy : 0.8514831213369847
    f1_score : 0.6433970380384686
    


Tagguer le nouveau modèle en staging

In [38]:
model_name = "20230118_rf_census"
stage = 'Staging'

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{stage}"
)

y_train_pred = model.predict(X_train)
print_scores(y_train_pred, y_train)

2023/01/25 10:20:25 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 2.2.1, required: cloudpickle==2.2.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.



    accuracy : 0.8514831213369847
    f1_score : 0.6433970380384686
    
